In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn")

In [6]:
#открываем файл с данными без пропусков и с добавленной колонкой с ценой за 1 кв.м
data_without_miss = pd.read_csv('data_without_miss.csv') 
data_without_miss

Округ Ближайшая станция метро  Время до метро  \
0                      СВАО            Марьина роща            17.0   
1                      СЗАО             Трикотажная             4.0   
2                       САО           Тимирязевская             3.0   
3                       ЗАО                    Фили             5.0   
4                       САО             Белорусская             4.0   
...                     ...                     ...             ...   
1095   НАО (Новомосковский)                Ольховая            17.0   
1096   НАО (Новомосковский)             Рассказовка             2.0   
1097                   СЗАО                 Спартак             5.0   
1098                    САО                 Водники            12.0   
1099                   СЗАО         Пятницкое шоссе             4.0   

      Метро рядом            Застройщик  \
0               2       Компания Пионер   
1               3           СК Резиденс   
2               3              MR Group   
3               3      Аеон Девелопмент   
4               3              MR Group   
...           ...                   ...   
1095            3         Группа Родина   
1096            3  Абсолют Недвижимость   
1097            3                   ПИК   
1098            3                 ДСК-1   
1099            3                   ПИК   

                                                     ЖК    Класс Тип квартиры  \
0                                       «PRIDE (Прайд)»  Премиум     квартира   
1                                         «ÁLIA (АЛИЯ)»   Бизнес     квартира   
2                            «SYMPHONY 34 (Симфони 34)»  Премиум     квартира   
3     «River Park Towers Кутузовский (Ривер Парк Тау...  Премиум     квартира   
4                 «Комплекс апартаментов Slava (Слава)»  Премиум  апартаменты   
...                                                 ...      ...          ...   
1095  «Russian Design District (Рашен Дизайн Дистрикт)»   Бизнес     квартира   
1096                   «Переделкино Ближнее Город-парк»  Комфорт     квартира   
1097                      «Holland park (Холланд парк)»  Комфорт     квартира   
1098                             «Первый Ленинградский»  Комфорт     квартира   
1099                                    «Митинский лес»  Комфорт     квартира   

      Этаж  Этажей в доме       Отделка  Количество комнат  Площадь квартиры  \
0      5.0           22.0   Без отделки                1.0             32.20   
1      2.0           26.0    Неизвестно                1.0             48.06   
2      2.0           36.0      Чистовая                1.0             37.62   
3      2.0           45.0   Без отделки                1.0             42.80   
4      2.0           16.0  Предчистовая                1.0             47.90   
...    ...            ...           ...                ...               ...   
1095  12.0           13.0   Без отделки                1.0             36.90   
1096   1.0            9.0   Без отделки                2.0             52.40   
1097   2.0           13.0   Без отделки                1.0             34.90   
1098   2.0           14.0  Предчистовая                2.0             52.90   
1099   6.0           32.0      Чистовая                2.0             50.80   

      Жилая площадь  Площадь кухни               Тип дома  \
0         20.333629      11.724697             Монолитный   
1         20.333629      11.724697    Монолитно-кирпичный   
2         12.800000       3.100000             Монолитный   
3         20.333629      11.724697    Монолитно-кирпичный   
4         20.333629      11.724697    Монолитно-кирпичный   
...             ...            ...                    ...   
1095      12.200000      11.400000    Монолитно-кирпичный   
1096      26.300000      11.724697  Панельный, монолитный   
1097      10.600000      16.100000             Монолитный   
1098      25.500000      10.600000              Панельный   
1099      20.600000      17.900000             

## Гипотеза о "равенстве математических ожиданий цен на апартаменты/апартаменты-студии и другие типы квартир"

Для определения подходящей статистики проведем дополнительное исследование на нормальность сравниваемых нами выборок с помощью ***теста Шапиро-Уилка*** (https://www.codecamp.ru/blog/shapiro-wilk-test-python/):

**Тест Шапиро-Уилка**



$$
\begin{aligned}
& H_0: Данные \ выборки \ имеют \ нормальное \ распределение \\
& H_1: Данные \ выборки \ НЕ \ имеют \ нормальное \ распределение
\end{aligned}
$$

In [30]:
apartments = data_without_miss[(data_without_miss['Тип квартиры'] == 'апартаменты') | (data_without_miss['Тип квартиры'] == 'Апартаменты-студия')]['Цена за 1 кв.м']
other = data_without_miss[(data_without_miss['Тип квартиры'] == 'квартира') | (data_without_miss['Тип квартиры'] == 'Студия')]['Цена за 1 кв.м']
from scipy import stats
shapiro_apartments = stats.shapiro(apartments)
shapiro_other = stats.shapiro(other)
print(f'P-value апартаментов/апартаментов-студий: {shapiro_apartments[1]}')
print(f'P-value других квартир: {shapiro_other[1]}')

P-value апартаментов/апартаментов-студий: 3.480594447640567e-13
P-value других квартир: 1.4073258858366863e-17


Исходя из результатов данного теста, можно сделать вывод о том, что у нас достаточно доказательств для отвержения нулевой гипотезы — *данные обеих выборок не имеют нормального распределения*, потому что значения их p-value гораздо меньше 0.05.

**Критерий Уилкоксона**

Так как нормальность выборок не подтвердилась, а значит предпосылки для использования t-статистики не были выполнены, то прибегнем к использованию ***Критерия Уилкоксона*** (в силу того, что наши выборки зависимы). 

Для начала проверим, что размеры исследуемых выборок малы (<25) (https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%B8%D1%82%D0%B5%D1%80%D0%B8%D0%B9_%D0%A3%D0%B8%D0%BB%D0%BA%D0%BE%D0%BA%D1%81%D0%BE%D0%BD%D0%B0#%D0%9E%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%87%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BA%D1%80%D0%B8%D1%82%D0%B5%D1%80%D0%B8%D1%8F): 

In [10]:
n = len(apartments)
m = len(other)
print(f'Длина выборки апартаментов/апартаментов-студий: {n}')
print(f'Длина выборки других квартир: {m}')

Длина выборки апартаментов/апартаментов-студий: 114
Длина выборки других квартир: 664


Уверенно критерий Уилкоксона можно использовать при объёме выборки до 25 элементов, потому что в случае большего числа наблюдений распределение значений данного критерия стремительно приближается к нормальному. У нас же объёмы обеих выборок явно больше требуемого значения.Вот почему становится необходимым прибегнуть к преобразованию критерия Уилкоксона в величину ***z (z-score)*** (https://www.codecamp.ru/blog/z-score-python/). 

**Z-score**

Для проверки гипотезы о том, что  𝜇
  цен апартаментов равно  𝜇
  цен обычных квартир, воспользуемся z-статистикой, так как мы не знаем дисперсии наших данных.

Сформулируем гипотезы:

$$
\begin{aligned}
& H_0: \mu_{apartments} = \mu_{other} \\
& H_1: \mu_{apartments} \neq \mu_{other}
\end{aligned}
$$

Уровень значимости: $5\%$

*Формула z-статистики:*

$$
z = \frac {\bar{X} - \bar{Y} - \mu_0}{\sqrt{\frac {\hat{\sigma}_X^2}{n} + {\frac {\hat{\sigma}_Y^2}{m}}}}
$$

In [14]:
mean_apartments = apartments.mean()
mean_other = other.mean()
n = len(apartments)
m = len(other)
import statistics as st
var_apartments = st.variance(apartments)
var_other = st.variance(other)
z_test = (mean_apartments - mean_other - 0)/(np.sqrt(var_apartments/n + var_other/m))
z_test

-0.4175545840930239

In [15]:
alpha = 0.05
from scipy import stats
z_crit = stats.norm.ppf(1-alpha/2)
if z_test < -z_crit or z_test > z_crit:
    print("H0 отвергается")
else:
    print("Нет оснований отвергать H0")

Нет оснований отвергать H0


**Вывод**: Мы видим, что полученное нами значение меньше критического значения Z, поэтому результаты теста являются статистически не значимыми. Вот почему у нас нет оснований для того, чтобы отвергнуть нулевую гипотезу, а значит математическое ожидание цен апартаментов/апартаментов-студий действительно равно математическому ожиданию цен других квартир, то есть их средние цены одинаковые.

## Гипотеза о "равенстве математических ожиданий цен на апартаменты и обычные квартиры"

Для определения нормальности распределения выборок теперь воспользуемся ***тестом Колмогорова-Смирнова*** (https://www.codecamp.ru/blog/kolmogorov-smirnov-test-python/):

**Тест Колмогорова-Смирнова**



$$
\begin{aligned}
& H_0: Данные \ выборки \ имеют \ нормальное \ распределение \\
& H_1: Данные \ выборки \ НЕ \ имеют \ нормальное \ распределение
\end{aligned}
$$

In [32]:
apartment = data_without_miss[data_without_miss['Тип квартиры'] == 'апартаменты']['Цена за 1 кв.м']
ordinary = data_without_miss[data_without_miss['Тип квартиры'] == 'квартира']['Цена за 1 кв.м']
mean_apartment = apartment.mean()
mean_ordinary = ordinary.mean()
std_apartment = apartment.std(ddof=1)
std_ordinary = ordinary.std(ddof=1)
ks_apartment = stats.kstest(apartment, 'norm', args=(mean_apartment, std_apartment))
ks_ordinary = stats.kstest(ordinary, 'norm', args=(mean_ordinary, std_ordinary))
print(f'P-value апартаментов: {ks_apartment[1]}')
print(f'P-value обычных квартир: {ks_ordinary[1]}')

P-value апартаментов: 0.01459675665042759
P-value обычных квартир: 0.0008681066342900275


Мы видим, что у нас достаточно доказательств для отвержения нулевой гипотезы — *данные обеих выборок не имеют нормального распределения* (p-value меньше 0.05).

**Критерий Уилкоксона**

Так как нормальность выборок не подтвердилась, а значит предпосылки для использования t-статистики не были выполнены, прибегнем к использованию ***Критерия Уилкоксона*** (в силу того, что наши выборки зависимы). 

Снова проверим их объем (<25): 

In [72]:
n = len(apartment)
m = len(ordinary)
print(f'Длина выборки апартаментов: {n}')
print(f'Длина выборки обычных квартир: {m}')

Длина выборки апартаментов: 114
Длина выборки обычных квартир: 664


Мы понимаем, что полученные значения явно больше 25 — это вынуждает нас вновь использовать ***z (z-score)***. 

**Z-score**

Для проверки гипотезы о том, что  𝜇
  цен апартаментов равно  𝜇
  цен обычных квартир, воспользуемся z-статистикой, так как мы не знаем дисперсий наших данных.

Сформулируем гипотезы:

$$
\begin{aligned}
& H_0: \mu_{apartment} = \mu_{ordinary} \\
& H_1: \mu_{apartment} \neq \mu_{ordinary}
\end{aligned}
$$

Уровень значимости: $5\%$

*Формула z-статистики:*

$$
z = \frac {\bar{X} - \bar{Y} - \mu_0}{\sqrt{\frac {\hat{\sigma}_X^2}{n} + {\frac {\hat{\sigma}_Y^2}{m}}}}
$$

In [58]:
mean_apartment = apartment.mean()
mean_ordinary = ordinary.mean()
n = len(apartment)
m = len(ordinary)
import statistics as st
var_apartment = st.variance(apartment)
var_ordinary = st.variance(ordinary)
z_test = (mean_apartment - mean_ordinary - 0)/(np.sqrt(var_apartment/n +var_ordinary/m))
z_test

-0.4175545840930239

In [59]:
alpha = 0.05
from scipy import stats
z_crit = stats.norm.ppf(1-alpha/2)
if z_test < -z_crit or z_test > z_crit:
    print("H0 отвергается")
else:
    print("Нет оснований отвергать H0")

Нет оснований отвергать H0


**Вывод**: Мы видим, что полученное нами значение больше левостороннего критического значения Z, поэтому результаты теста являются статистически не значимыми. Вот почему у нас нет оснований для того, чтобы отвергнуть нулевую гипотезу, а значит математическое ожидание цен апартаментов действительно равно математическому ожиданию цен обычных квартир, то есть их средние цены одинаковые.

## Гипотеза о "равенстве математических ожиданий цен на квартиры с отделкой и без неё"

Для определения подходящей статистики опять проведем дополнительное исследование на нормальность сравниваемых нами выборок с помощью ***теста Шапиро-Уилка***:

**Тест Шапиро-Уилка**



$$
\begin{aligned}
& H_0: Данные \ выборки \ имеют \ нормальное \ распределение \\
& H_1: Данные \ выборки \ НЕ \ имеют \ нормальное \ распределение
\end{aligned}
$$

In [33]:
fine_finish = data_without_miss[data_without_miss['Отделка'] == 'Чистовая']['Цена за 1 кв.м']
without_finishing = data_without_miss[data_without_miss['Отделка'] == 'Без отделки']['Цена за 1 кв.м']
shapiro_fine_finish = stats.shapiro(fine_finish)
shapiro_without_finishing = stats.shapiro(without_finishing)
print(f'P-value квартир с отделкой: {shapiro_fine_finish[1]}')
print(f'P-value квартир без отделки: {shapiro_without_finishing[1]}')

P-value квартир с отделкой: 6.597828986547383e-25
P-value квартир без отделки: 2.4431611223008076e-07


Исходя из результатов данного теста, можно сделать вывод о том, что у нас достаточно доказательств для отвержения нулевой гипотезы — *данные обеих выборок не имеют нормального распределения*, потому что значения их p-value гораздо меньше 0.05.

**Критерий Уилкоксона**

Так как нормальность выборок не подтвердилась, а значит предпосылки для использования t-статистики не были выполнены, то прибегнем к использованию ***Критерия Уилкоксона*** (в силу того, что наши выборки зависимы). 

Для начала проверим, что размеры исследуемых выборок малы (<25): 

In [20]:
n = len(fine_finish)
m = len(without_finishing)
print(f'Длина выборки квартир с чистовой отделкой: {n}')
print(f'Длина выборки квартир без отделки: {m}')

Длина выборки квартир с чистовой отделкой: 444
Длина выборки квартир без отделки: 389


Полученные значения оказались больше 25, следовательно, переходим к ***z (z-score)***. 

**Z-score**

Для того чтобы подтвердить или опровергнуть гипотезу о том, что  𝜇
  цен квартир с отделкой равно  𝜇
  цен квартир без отделки, воспользуемся z-статистикой, так как дисперсии выборок нам неизвестны.

Сформулируем гипотезы:

$$
\begin{aligned}
& H_0: \mu_{с \ отделкой} = \mu_{без \ отделки} \\
& H_1: \mu_{с \ отделкой} \neq \mu_{без \ отделки}
\end{aligned}
$$

Уровень значимости: $5\%$

*Формула z-статистики:*

$$
z = \frac {\bar{X} - \bar{Y} - \mu_0}{\sqrt{\frac {\hat{\sigma}_X^2}{n} + {\frac {\hat{\sigma}_Y^2}{m}}}}
$$

Возьмем квартиры "без отделки" и с "чистовой" отделкой", так как "предчистовая" отделка является промежуточным вариантом и нельзя точно сказать, на сколько в дальнейшем вырастет цена полностью отделанной квартиры.

In [21]:
mean_fine_finish = fine_finish.mean()
mean_without_finishing = without_finishing.mean()
n = len(fine_finish)
m = len(without_finishing)
import statistics as st
var_fine_finish = st.variance(fine_finish)
var_without_finishing = st.variance(without_finishing)
z_test = (mean_fine_finish - mean_without_finishing - 0)/(np.sqrt(var_fine_finish/n +var_without_finishing/m))
z_test

-8.862186715477845

In [22]:
alpha = 0.05
from scipy import stats
z_crit = stats.norm.ppf(1-alpha/2)
if z_test < -z_crit or z_test > z_crit:
    print("H0 отвергается")
else:
    print("Нет оснований отвергать H0")

H0 отвергается


**Вывод**: Полученное значение больше левостороннего критического значения Z, поэтому результаты теста являются статистически значимыми. Следовательно, нулевая гипотеза о том, что средняя цена квартиры с отделкой равна средней цене квартиры без отделки отвергается в пользу альтернативной. 